In [14]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

messages=Queue()
recordings=Queue()
record_button=widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone",
)

stop_button=widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop",
)

output=widgets.Output()
def start_recording(data):
    messages.put(True)
    with output:
        display("Starting....")
        record=Thread(target=record_microphone)
        record.start()
        transcribe=Thread(target=speech_recognition,args=(output,))
        transcribe.start()
        f=open("text.txt","w")
        f.write(transcribe)
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped....")
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button,stop_button,output)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

Speech Recognition Functions

In [15]:
import pyaudio
p=pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()    

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (4- USB Audio Device', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Speakers (4- USB Audio Device)', 'hostApi': 0, 'maxInputChanne

In [16]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 8
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=1,
                    frames_per_buffer=chunk)

    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [17]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time


model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)
    
def speech_recognition(output):
    
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        tokenizer = T5Tokenizer.from_pretrained('SJ-Ray/Re-Punctuate')
        model = TFT5ForConditionalGeneration.from_pretrained('SJ-Ray/Re-Punctuate')
        input_text = text
        inputs = tokenizer.encode("punctuate: " + input_text, return_tensors="tf") 
        result = model.generate(inputs)
        decoded_output = tokenizer.decode(result[0], skip_special_tokens=True)
        output.append_stdout(decoded_output)
        


        
        


        
     
       

 
